# Date Night Movie

#### Grading:


- Code: 90 pts
- Markdown Documentation: 10 pts


In this assignment we are going to use pandas to figure out - What's the best **date-night movie**?

This assignment is going to use
- Joining
- Groupby
- Sorting


In [2]:
import os
import pandas as pd

##### Read in the movie data: `pd.read_table`

In [3]:
def get_movie_data():
    
    unames = ['user_id','gender','age','occupation','zip']
    users = pd.read_table(os.path.join('../data','users.dat'), 
                          sep='::', header=None, names=unames)
    
    rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
    ratings = pd.read_table(os.path.join('../data', 'ratings.dat'), 
                            sep='::', header=None, names=rnames)
    
    mnames = ['movie_id', 'title','genres']
    movies = pd.read_table(os.path.join('../data', 'movies.dat'), 
                           sep='::', header=None, names=mnames)

    return users, ratings, movies

In [4]:
users, ratings, movies = get_movie_data()

/home/christopherbrantley/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """
/home/christopherbrantley/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':
/home/christopherbrantley/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifyi

In [5]:
print users.head()

SyntaxError: invalid syntax (<ipython-input-5-363f407e1304>, line 1)

In [ ]:
print ratings.head()

In [ ]:
print movies.head()

##### Clean up the `movies`

- Get the `year`
- Shorten the `title`


In [ ]:
tmp = movies.title.str.extract('(.*) \(([0-9]+)\)')
tmp.apply(lambda x:x[0] if len(x) > 0 else None)
tmp.apply(lambda x: x[0][:40] if len(x) > 0 else None)

In [ ]:
movies['year'] = tmp[1]
movies['short_title'] = tmp[0]

In [ ]:
print movies.head()

##### Join the tables with `pd.merge` (10 pts)

In [28]:
import numpy as np
df_user_rating = pd.merge(users, ratings, how = "inner", on = "user_id")
df_user_rating_movie = pd.merge(df_user_rating, movies, how = "inner",\
                                 on = "movie_id")
df_user_rating_movie

,user_id,gender,age,occupation,zip,movie_id,rating,timestamp,title,genres
0,1,F,1,10,48067,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,M,56,16,70072,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,M,25,12,32793,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,M,25,7,22903,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,M,50,1,95350,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...,...,...,...,...
1000204,5949,M,18,17,47901,2198,5,958846401,Modulations (1998),Documentary
1000205,5675,M,35,14,30030,2703,3,976029116,Broken Vessels (1998),Drama
1000206,5780,M,18,17,92886,2845,1,958153068,White Boys (1999),Drama
1000207,5851,F,18,20,55410,3607,5,957756608,One Little Indian (1973),Comedy|Drama|Western


##### What's the highest rated movie? (10 pts))

In [48]:
#rating.groupby(by = 0)
for x in df_user_rating_movie.groupby(by = "movie_id").rating.sum():
    print(x)


8613
2244
1442
464
890
3646
1562
205
271
3144
3919
378
323
542
359
2587
3363
524
965
406
4914
1266
360
1984
3578
353
179
726
1637
270
439
5962
12
6814
232
3673
28
79
4935
118
958
634
572
867
1863
516
4669
1137
101
8054
1569
38
105
138
18
337
2051
22
1147
201
2000
317
179
295
162
13
207
1166
2885
202
338
855
357
23
508
128
163
297
199
525
345
122
65
660
801
168
565
704
22
206
199
678
1835
51
165
12
173
392
979
109
106
2405
1251
43
806
20
10346
5188
1963
158
34
168
287
209
32
8
150
537
445
45
1219
197
1
52
43
8
49
227
1
8
374
26
39
5
4
440
2421
1
1469
1193
52
641
64
178
5096
1988
207
2053
626
297
235
511
772
381
1265
2830
1995
1867
1352
2938
150
18
1068
228
1010
332
1180
1302
187
765
803
380
66
106
1600
150
154
101
87
1633
583
340
612
51
375
185
6
445
800
466
1553
52
1994
446
29
87
75
620
593
307
301
686
1713
128
129
121
27
499
119
609
1130
168
610
70
51
1167
5573
1309
806
2
345
28
378
1205
2107
1410
849
327
3427
1077
764
90
483
122
96
280
21
19
33
2999
1844
402
981
78
59
1200
2601
117
1

247
1721
376
116
602
52
700
1218
2795
458
2702
2068
4686
9778
76
1505
595
123
1176
1133
1418
705
2129
4962
3559
655
1414
1153
992
576
1439
1285
42
1115
218
338
221
1685
1277
607
1848
2833
210
80
2729
1425
509
448
1487
458
2056
218
305
438
310
4
715
158
79
612
453
20
207
160
814
444
49
911
299
32
572
1612
3685
657
876
322
796
273
130
99
1644
21
85
45
1264
1036
1658
3125
1088
277
465
1187
462
1004
96
1774
100
9
126
100
73
2
1251
16
17
328
2710
315
397
116
108
353
1211
482
287
184
309
2433
3332
42
522
779
545
161
50
11
317
75
903
239
93
83
836
1021
310
563
221
186
97
803
342
310
2176
2172
4314
1075
884
1018
1408
43
339
70
66
18
2091
473
1801
2436
7
50
34
497
234
390
641
622
82
735
432
279
3
17
645
70
532
398
34
3
1471
97
967
451
34
367
11178
2396
164
531
564
6
92
14
110
2787
1977
205
848
3
185
74
302
175
146
4
3
130
245
503
167
1608
5982
1335
17
2462
564
278
13
283
159
571
556
870
85
569
1674
3345
179
3
133
252
1390
11
417
203
125
48
7672
396
144
10
36
573
302
94
32
88
102
522
4322
2677
1

###### What is a good rated movie for date night? (30 pts)

- Hint - highly rated movie by 
    - both partners (might be the same gender or not),
    - based on genre preferences,
    - age group can also be combined